In [ ]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 3.4 MB 41.4 MB/s 
     |████████████████████████████████| 45 kB 2.9 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=be9ab701e58413a49dc3f357d8150074e901553ae45659efcc6e610e728ec427
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
Successfully built parsimonious


In [ ]:
from pysat.formula import IDPool
Atom = IDPool()
from pysat.formula import CNF
from pysat.solvers import Cadical


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving column_bin_3a_5p.csv to column_bin_3a_5p.csv


In [235]:
import pandas as pd
import io
 
arquivo = pd.read_csv(io.BytesIO(uploaded['column_bin_3a_5p.csv']))
arquivo.index += 1
arquivo

,PI <= 42.09,LA <= 39.63,GS <= 37.89,P
1,0,0,0,1
2,0,1,0,1
3,1,1,1,0
4,0,0,1,0
5,1,0,0,0


In [236]:
#Quantidade de pacientes do DataFrame
qtdPacientes=arquivo.shape[0]
print("Quantidade de Pacientes: ",qtdPacientes)

#Atributos do DataFrame
lista_atributos=arquivo.columns.tolist()
print("Atributos: ",lista_atributos)

#Quantidade de atributos do DataFrame
print("Quantidade de Atributos: ",arquivo.shape[1])

#Valoração
valoracao= [tuple(x) for x in arquivo.values]
print(valoracao)

#Quantidade de Regras
m=4



Quantidade de Pacientes:  5
Atributos:  ['PI <= 42.09', 'LA <= 39.63', 'GS <= 37.89', 'P']
Quantidade de Atributos:  4
[(0, 0, 0, 1), (0, 1, 0, 1), (1, 1, 1, 0), (0, 0, 1, 0), (1, 0, 0, 0)]


## **Restrição 1** 

In [237]:
def restricao1(m, lista_atributos):

    lista = []
    lista2 = []
    formula = []
    for regra in range(m):
        for atributo in lista_atributos:
            if atributo != 'P':
                lista=[]
                lista.append(Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 's'))
                lista2.append(lista)

                lista=[]
                lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                lista.append(Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 's'))
                lista2.append(lista)

                lista=[]
                lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                lista.append(Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 's'))
                lista2.append(lista)

    return lista2

r1= restricao1(m, lista_atributos)
print(r1)

[[1, -2, -3], [-1, 2, -3], [-1, -2, 3], [40, -41, -42], [-40, 41, -42], [-40, -41, 42], [309, -310, -311], [-309, 310, -311], [-309, -310, 311], [61, -62, -63], [-61, 62, -63], [-61, -62, 63], [100, -101, -102], [-100, 101, -102], [-100, -101, 102], [312, -313, -314], [-312, 313, -314], [-312, -313, 314], [121, -122, -123], [-121, 122, -123], [-121, -122, 123], [160, -161, -162], [-160, 161, -162], [-160, -161, 162], [315, -316, -317], [-315, 316, -317], [-315, -316, 317], [181, -182, -183], [-181, 182, -183], [-181, -182, 183], [220, -221, -222], [-220, 221, -222], [-220, -221, 222], [318, -319, -320], [-318, 319, -320], [-318, -319, 320]]


## **Restrição 2** 

In [238]:
def restricao2(m, lista_atributos):
    lista1 = []
    regra=0
    while regra < m:
        lista2 = []
        for atributo in range(0, len(lista_atributos) - 1):
            lista2.append(-Atom.id(str(lista_atributos) + '-' + str(regra + 1) + '-' + 'n'))
        lista1.append(lista2)
        regra+=1;
    return(lista1)

r2= restricao2(m, lista_atributos)
print(r2)

[[-321, -321, -321], [-322, -322, -322], [-323, -323, -323], [-324, -324, -324]]


## **Restrição 3** 

In [239]:
def restricao3(m, lista_atributos, qtdPacientes, valoracao):
    lista = []
    lista2 = []
    j=0
    regra=0
    while True:
      if j == qtdPacientes:
          break
      if valoracao[j][lista_atributos.index('P')] == 0:
          while True:
            if regra == m:
              break;
            lista = []
            for a in lista_atributos:
                  if a != 'P':
                    if valoracao[j][lista_atributos.index(str(a))] == 1: 
                        lista.append(-Atom.id(str(lista_atributos) + '-' + str(regra + 1) + '-' + 'n'))
                    elif valoracao[j][lista_atributos.index(str(a))] == 0:
                        lista.append(-Atom.id(str(lista_atributos) + '-' + str(regra + 1) + '-' + 'p'))
            regra+=1
            lista2.append(lista)
      j+=1
    return lista2

r3= restricao3(m, lista_atributos, qtdPacientes, valoracao)
print(r3)

[[-321, -321, -321], [-322, -322, -322], [-323, -323, -323], [-324, -324, -324]]


## **Restrição 4** 

In [243]:
def restricao4(m, qtdPacientes, lista_atributos, valoracao):
    lista = []
    lista2 = []

    for j in range(qtdPacientes):
        if valoracao[j][lista_atributos.index('P')] == 1:
            for regra in range(m):     
                for atributo in lista_atributos:
                    lista = []
                    if atributo != 'P':
                        if valoracao[j][lista_atributos.index(str(atributo))] == 1:
                            lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                            lista.append(-Atom.id('C-' + str(regra+1) + '-'+str(j+1)))
                        else:
                            lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                            lista.append(-Atom.id('C-' + str(regra+1) + '-'+str(j+1)))
                        lista2.append(lista)

    return lista2

r4=restricao4(m ,qtdPacientes, lista_atributos, valoracao)
print(r4)

[[-1, -249], [-40, -249], [-309, -249], [-61, -250], [-100, -250], [-312, -250], [-121, -251], [-160, -251], [-315, -251], [-181, -252], [-220, -252], [-318, -252], [-1, -253], [-41, -253], [-309, -253], [-61, -254], [-101, -254], [-312, -254], [-121, -255], [-161, -255], [-315, -255], [-181, -256], [-221, -256], [-318, -256]]


## **Restrição 5** 

In [241]:
def restricao5(m, qtdPacientes, valoracao, lista_atributos):
    lista = []
    lista2 = []
    for j in range(qtdPacientes):
        lista = []
        if valoracao[j][lista_atributos.index('P')] == 1:
          for regra in range(m):
              lista.append(Atom.id('C-' + str(regra+1) + '-'+str(j+1)))
          lista2.append(lista)
    formula = lista2
    return formula
r5= restricao5(m, qtdPacientes, valoracao, lista_atributos)
print(r5)

[[249, 250, 251, 252], [253, 254, 255, 256]]


## **Teste** 

In [242]:

teste = restricao1(m, lista_atributos)+restricao2(m, lista_atributos)+restricao3(m, lista_atributos, qtdPacientes, valoracao)+restricao4(m ,qtdPacientes, lista_atributos, valoracao)+restricao5(m, qtdPacientes, valoracao, lista_atributos)
print(teste)


[[1, -2, -3], [-1, 2, -3], [-1, -2, 3], [40, -41, -42], [-40, 41, -42], [-40, -41, 42], [309, -310, -311], [-309, 310, -311], [-309, -310, 311], [61, -62, -63], [-61, 62, -63], [-61, -62, 63], [100, -101, -102], [-100, 101, -102], [-100, -101, 102], [312, -313, -314], [-312, 313, -314], [-312, -313, 314], [121, -122, -123], [-121, 122, -123], [-121, -122, 123], [160, -161, -162], [-160, 161, -162], [-160, -161, 162], [315, -316, -317], [-315, 316, -317], [-315, -316, 317], [181, -182, -183], [-181, 182, -183], [-181, -182, 183], [220, -221, -222], [-220, 221, -222], [-220, -221, 222], [318, -319, -320], [-318, 319, -320], [-318, -319, 320], [-321, -321, -321], [-322, -322, -322], [-323, -323, -323], [-324, -324, -324], [-321, -321, -321], [-322, -322, -322], [-323, -323, -323], [-324, -324, -324], [-1, -249], [-40, -249], [-309, -249], [-61, -250], [-100, -250], [-312, -250], [-121, -251], [-160, -251], [-315, -251], [-181, -252], [-220, -252], [-318, -252], [-1, -253], [-41, -253], [-